In [7]:
%load_ext autoreload
%autoreload 2

In [ ]:
import boto3
from botocore.config import Config

from DataFin import FMPClient, S3Client

import os
import dotenv
import boto3
import json
import pandas as pd
import io

dotenv.load_dotenv()

fmp_api_key = os.getenv('FMP_API_KEY')

s3_bucket = os.getenv('PERSONAL_S3_BUCKET_NAME')
aws_access_key =  os.getenv('PERSONAL_AWS_ACCESS_KEY_ID')
aws_secret_key =  os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY_ID')


fmp = FMPClient(fmp_api_key)

personal_s3 = S3Client(
    aws_account_access_key = aws_access_key,
    aws_secret_account_access_key = aws_secret_key,
    region = 'us-east-1',
    bucket_name = s3_bucket
)

polygon_aws_access_key = os.getenv('POLYGON_AWS_ACCESS_KEY_ID')
polygon_aws_secret_key = os.getenv('POLYGON_AWS_SECRET_ACCESS_KEY_ID')


session = boto3.Session(
   aws_access_key_id = polygon_aws_access_key,
   aws_secret_access_key = polygon_aws_secret_key
)

s3 = session.client(
   's3',
   endpoint_url='https://files.polygon.io',
   config=Config(signature_version='s3v4'),
)



from datetime import datetime, timedelta
now = datetime.now()
yesterday = now - timedelta(days=1)

year = now.year  
month = f'{now.month:02d}'
day = f'{now.day:02d}'
yesterday = f'{yesterday.day:02d}'
print(month, day, yesterday)


bucket_name = 'flatfiles'
object_key = f'us_stocks_sip/minute_aggs_v1/2025/{month}/2025-{month}-{yesterday}.csv.gz'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)


df = pd.read_csv(io.BytesIO(obj['Body'].read()), compression='gzip')

04 22 21


,ticker,volume,open,close,high,low,window_start,transactions
0,A,21298,101.850,101.9850,102.040,101.850,1745242200000000000,100
1,A,2517,101.750,101.7500,101.750,101.750,1745242320000000000,12
2,A,4292,101.750,101.5550,101.750,101.430,1745242380000000000,72
3,A,8285,101.320,101.3700,101.380,100.580,1745242440000000000,138
4,A,3856,101.370,101.3125,101.370,101.090,1745242500000000000,27
...,...,...,...,...,...,...,...,...
1609341,ZYXI,1179,2.140,2.1450,2.150,2.140,1745265300000000000,10
1609342,ZYXI,403,2.140,2.1400,2.140,2.140,1745265360000000000,11
1609343,ZYXI,388,2.135,2.1350,2.135,2.135,1745265480000000000,6
1609344,ZYXI,1729,2.130,2.1400,2.140,2.130,1745265540000000000,37


In [9]:
personal_s3 = S3Client(
    aws_account_access_key = aws_access_key,
    aws_secret_account_access_key = aws_secret_key,
    region = 'us-east-1',
    bucket_name = s3_bucket
)

In [11]:
df['date_time'] = pd.to_datetime(df['window_start'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None)

df.to_parquet(f's3://data-fin-data-lake-test-1/dev/2025-04-{yesterday}-test.parquet')

OSError: When initiating multiple part upload for key 'dev/2025-04-21-test.parquet' in bucket 'data-fin-data-lake-test-1': AWS Error ACCESS_DENIED during CreateMultipartUpload operation: Anonymous users cannot initiate multipart uploads.  Please authenticate.

In [ ]:
df.to_csv('2025-04-21-test.csv')

In [3]:
df_p = pd.read_parquet('2025-04-21-test.parquet')
df_p

,ticker,volume,open,close,high,low,window_start,transactions,date_time
0,A,21298,101.850,101.9850,102.040,101.850,1745242200000000000,100,2025-04-21 09:30:00
1,A,2517,101.750,101.7500,101.750,101.750,1745242320000000000,12,2025-04-21 09:32:00
2,A,4292,101.750,101.5550,101.750,101.430,1745242380000000000,72,2025-04-21 09:33:00
3,A,8285,101.320,101.3700,101.380,100.580,1745242440000000000,138,2025-04-21 09:34:00
4,A,3856,101.370,101.3125,101.370,101.090,1745242500000000000,27,2025-04-21 09:35:00
...,...,...,...,...,...,...,...,...,...
1609341,ZYXI,1179,2.140,2.1450,2.150,2.140,1745265300000000000,10,2025-04-21 15:55:00
1609342,ZYXI,403,2.140,2.1400,2.140,2.140,1745265360000000000,11,2025-04-21 15:56:00
1609343,ZYXI,388,2.135,2.1350,2.135,2.135,1745265480000000000,6,2025-04-21 15:58:00
1609344,ZYXI,1729,2.130,2.1400,2.140,2.130,1745265540000000000,37,2025-04-21 15:59:00
